<a href="https://colab.research.google.com/github/SiddhiMarri/PostureDetection/blob/main/model_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# prompt: divide the labels and images files in train, test, val for YOLO with ratio 7:1:2

import os
import random
import shutil

def split_data(images_dir, labels_dir, output_dir, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
    """
    Splits image and label data into train, validation, and test sets.

    Args:
        images_dir: Path to the directory containing images.
        labels_dir: Path to the directory containing labels.
        output_dir: Path to the directory where the split data will be saved.
        train_ratio: Proportion of data for training.
        val_ratio: Proportion of data for validation.
        test_ratio: Proportion of data for testing.
    """

    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Create subdirectories
    for subset in ["train", "val", "test"]:
        subset_dir = os.path.join(output_dir, subset)
        os.makedirs(os.path.join(subset_dir, "images"), exist_ok=True)
        os.makedirs(os.path.join(subset_dir, "labels"), exist_ok=True)

    image_files = [f for f in os.listdir(images_dir) if os.path.isfile(os.path.join(images_dir, f))]
    num_images = len(image_files)
    indices = list(range(num_images))
    random.shuffle(indices)

    train_split = int(num_images * train_ratio)
    val_split = int(num_images * (train_ratio + val_ratio))

    for i in indices:
        image_file = image_files[i]
        label_file = image_file[:-4] + ".txt" # Assuming labels have same name as images but .txt extension

        if i < train_split:
            subset = "train"
        elif i < val_split:
            subset = "val"
        else:
            subset = "test"

        image_src = os.path.join(images_dir, image_file)
        label_src = os.path.join(labels_dir, label_file)
        image_dst = os.path.join(output_dir, subset, "images", image_file)
        label_dst = os.path.join(output_dir, subset, "labels", label_file)

        shutil.copy(image_src, image_dst)
        if os.path.exists(label_src):
            shutil.copy(label_src, label_dst)
        else:
            print(f"Warning: Label file not found for {image_file}. Skipping copy.")
    print("Data split successfully!")

# Example usage:
images_dir = "/content/images" # Replace with your images directory
labels_dir = "/content/labels" # Replace with your labels directory
output_dir = "/content/data"  # Replace with your desired output directory
split_data(images_dir, labels_dir, output_dir)

Data split successfully!


In [ ]:
import os
def count_files(directory):
    """Counts the number of files in a given directory."""
    try:
        num_files = len([f for f in os.listdir(directory) if os.path.isfile(os.path.join(directory, f))])
        return num_files
    except FileNotFoundError:
        print(f"Error: Directory '{directory}' not found.")
        return 0
    except Exception as e:
        print(f"An error occurred: {e}")
        return 0

# Example usage:
directory_path = "/content/data/val/labels"  # Replace with your directory path
file_count = count_files(directory_path)

if file_count >= 0:
    print(f"Number of files in '{directory_path}': {file_count}")

Number of files in '/content/data/val/labels': 16


In [ ]:
import shutil
import os

def zip_folder(folder_path, zip_filename):
  try:
    shutil.make_archive(zip_filename, 'zip', folder_path)
    print(f"Folder '{folder_path}' zipped successfully to '{zip_filename}.zip'")
  except Exception as e:
    print(f"Error zipping folder: {e}")
zip_folder("/content/data","/content/data")


from google.colab import files
files.download("/content/data.zip")

Folder '/content/data' zipped successfully to '/content/data.zip'


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import os
import tensorflow as tf
label_dir = "/content/data/val/labels"
image_dir = "/content/data/val/images"

def fix_labels(label_dir, image_dir):
    for label_file in os.listdir(label_dir):
        label_path = os.path.join(label_dir, label_file)
        image_path = os.path.join(image_dir, os.path.splitext(label_file)[0] + ".jpg")

        if not os.path.exists(image_path):
            print(f"Image {image_path} not found, skipping...")
            continue

        # Load image dimensions
        img = tf.io.decode_jpeg(tf.io.read_file(image_path))
        height, width = img.shape[:2]

        # Process label file
        fixed_lines = []
        with open(label_path, "r") as f:
            for line in f:
                parts = line.strip().split()
                class_id = parts[0]
                coords = list(map(float, parts[1:]))

                # Extract bbox and keypoints
                x_center, y_center, box_width, box_height = coords[:4]
                keypoints = coords[4:]

                # Check and normalize bbox
                if not (0 <= x_center <= 1 and 0 <= y_center <= 1 and 0 <= box_width <= 1 and 0 <= box_height <= 1):
                    x_center = min(max(x_center, 0), 1)
                    y_center = min(max(y_center, 0), 1)
                    box_width = min(max(box_width, 0), 1)
                    box_height = min(max(box_height, 0), 1)

                # Check and normalize keypoints
                for i in range(0, len(keypoints), 3):
                    keypoints[i] = min(max(keypoints[i], 0), 1)  # x
                    keypoints[i + 1] = min(max(keypoints[i + 1], 0), 1)  # y
                    keypoints[i + 2] = min(max(keypoints[i + 2], 0), 1)  # visibility

                fixed_line = [class_id, x_center, y_center, box_width, box_height] + keypoints
                fixed_lines.append(" ".join(map(str, fixed_line)))

        # Write back fixed labels
        with open(label_path, "w") as f:
            f.write("\n".join(fixed_lines))

fix_labels(label_dir, image_dir)


In [ ]:
pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 887.4/887.4 kB 26.5 MB/s eta 0:00:00


In [ ]:
!yolo data=/content/data.yaml check

Ultralytics 8.3.35 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 32.6/107.7 GB disk)

OS                  Linux-6.1.85+-x86_64-with-glibc2.35
Environment         Colab
Python              3.10.12
Install             pip
RAM                 12.67 GB
Disk                32.6/107.7 GB
CPU                 Intel Xeon 2.20GHz
CPU count           2
GPU                 None
GPU count           None
CUDA                None

numpy               ✅ 1.26.4>=1.23.0
numpy               ✅ 1.26.4<2.0.0; sys_platform == "darwin"
matplotlib          ✅ 3.8.0>=3.3.0
opencv-python       ✅ 4.10.0.84>=4.6.0
pillow              ✅ 11.0.0>=7.1.2
pyyaml              ✅ 6.0.2>=5.3.1
requests            ✅ 2.32.3>=2.23.0
scipy               ✅ 1.13.1>=1.4.1
torch               ✅ 2.5.1+cu121>=1.8.0
torch               ✅ 2.5.1+cu121!=2.4.0,>=1.8.0; sys_platform == "win32"
torchvision         ✅ 0.20.1+cu121>=0.9.0
tqdm                ✅ 4.66.6>=4.64.0
psutil           

In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolo11n-pose.pt")  # load a pretrained model (recommended for training)

# Train the model
results = model.train(data="data.yaml", epochs=20, imgsz=640)

Ultralytics 8.3.35 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=pose, mode=train, model=yolo11n-pose.pt, data=data.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train6, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_widt

train: Scanning /content/data/train/labels.cache... 53 images, 0 backgrounds, 0 corrupt: 100%|██████████| 53/53 [00:00<?, ?it/s]

albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01, num_output_channels=3, method='weighted_average'), CLAHE(p=0.01, clip_limit=(1.0, 4.0), tile_grid_size=(8, 8))



val: Scanning /content/data/val/labels... 16 images, 0 backgrounds, 0 corrupt: 100%|██████████| 16/16 [00:00<00:00, 1033.70it/s]

val: New cache created: /content/data/val/labels.cache
Plotting labels to runs/pose/train6/labels.jpg... 


...

      20/20         0G       1.63      2.968     0.6038      1.452      1.672          5        640: 100%|██████████| 4/4 [00:41<00:00, 10.33s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:03<00:00,  3.20s/it]

                   all         16         16      0.786      0.688      0.699      0.292      0.597      0.562      0.426     0.0736



20 epochs completed in 0.251 hours.
Optimizer stripped from runs/pose/train6/weights/last.pt, 5.6MB
Optimizer stripped from runs/pose/train6/weights/best.pt, 5.6MB

Validating runs/pose/train6/weights/best.pt...
Ultralytics 8.3.35 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
YOLO11n-pose summary (fused): 257 layers, 2,654,632 parameters, 0 gradients, 6.6 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95)     Pose(P          R      mAP50  mAP50-95): 100%|██████████| 1/1 [00:02<00:00,  2.87s/it]


                   all         16         16      0.681      0.812      0.741      0.267      0.477      0.625       0.47     0.0982
Speed: 1.9ms preprocess, 154.7ms inference, 0.0ms loss, 0.5ms postprocess per image
Results saved to runs/pose/train6
